In [1]:
# -*- coding: utf-8 -*-

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parse
from time import sleep
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import random
import matplotlib.pyplot as plt
import math
% matplotlib inline

In [3]:
import lxml
import requests
import html5lib
pd.options.display.max_rows = 50

In [110]:
from bokeh.models import HoverTool, CategoricalColorMapper, OpenURL, TapTool, ResetTool
from bokeh.plotting import ColumnDataSource, figure, output_file, show, save, output_notebook
from bokeh.io import curdoc

In [5]:
table_pages = ['https://en.wikipedia.org/wiki/List_of_battles_before_301',
               'https://en.wikipedia.org/wiki/List_of_battles_301%E2%80%931300',
               'https://en.wikipedia.org/wiki/List_of_battles_1301%E2%80%931600',
               'https://en.wikipedia.org/wiki/List_of_battles_since_2001']

In [6]:
list_pages = ['https://en.wikipedia.org/wiki/List_of_battles_1601%E2%80%931800',
               'https://en.wikipedia.org/wiki/List_of_battles_1801%E2%80%931900',
               'https://en.wikipedia.org/wiki/List_of_battles_1901%E2%80%932000',]

In [7]:
def list_battles(url):
    req = requests.get(url)
    print req.status_code
    era = req.content
    era_soup = BeautifulSoup(era)
    bullets = era_soup.find_all('li')
    
    for bullet in bullets:
        print bullet
        if 'title' in str(bullet):
#             if 'expedition' or 'recapture' or 'capture' or 'war' or 'action' or 'battle' or 'siege' or 'operation' or 'fall' in str(bullet).lower():
#                 if 'List+of+battles' not in str(bullet):
                    if 'footer' not in str(bullet):
                        link = bullet.a['href']
                        link_list.append(link)

                        title = bullet.a['title']
                        title_list.append(title)
                    else:
                        print 'NEXT ENTRY: ' + str(bullet)
#                 else:
#                     print 'NEXT ENTRY: ' + str(bullet)
#             else:
#                 print 'NEXT ENTRY: ' + str(bullet)
        else: 
            print 'NEXT ENTRY' + str(bullet)
                    

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(int req.status_code)? (<ipython-input-7-17f2e63ac4c1>, line 3)

In [ ]:
title_list = []
link_list = []
for url in list_pages:
    list_battles(url)

In [ ]:
link_list

In [ ]:
df_late_battles = pd.concat([pd.Series(link_list), pd.Series(title_list)], axis = 1).drop_duplicates()

In [ ]:
df_late_battles = df_late_battles.rename(columns = {0:'url', 1:'Battle'})

In [ ]:
df_late_battles

In [ ]:
df_late_battles[~df_late_battles['Battle'].str.contains('Battle' or 'Operation')]

In [ ]:
df_late_battles.to_csv('battle_list_1600-', encoding = 'utf-8')

In [ ]:
df_late_battles = pd.read_csv('battle_list_1600-.csv', encoding = 'utf-8')

In [ ]:
df_late_battles.head()

In [ ]:
df_late_battles = df_late_battles.drop(['Column1', 'Column4'], axis = 1)
df_late_battles.columns = df_late_battles.iloc[0]
df_late_battles = df_late_battles.drop(0)

In [ ]:
battle_list = []
link_list = []

In [ ]:
def early_battles(link):
    req = requests.get(link)
    print req.status_code
    era = req.content
    era_soup = BeautifulSoup(era)
    tables = era_soup.find_all('table', {'class': 'wikitable'})

    for table in tables:
        cells = table.find_all('td')
        for cell in cells:
            if cell.a is not None:
                a = ['Fall', 'Battle', 'Siege', 'Capture', 'Operation', 'Action', 'Recapture']
                if any(x in str(cell.a['href']) for x in a):
                    battle_list.append(cell.a['title'])
                    link_list.append(cell.a['href'])

In [ ]:
for page in table_pages:
    print page
    early_battles(page)

In [ ]:
df_battles = pd.DataFrame({'battle': battle_list,
                          'link':link_list})

In [ ]:
df_battles = df_battles.rename(columns = {'link':'url', 'battle':'Battle'})

In [ ]:
len(df_battles)

In [ ]:
len(df_late_battles)

In [ ]:
df_battles.head()

In [ ]:
df_url = pd.concat([df_late_battles, df_battles]).reset_index(drop = True)

In [ ]:
len(df_url)

In [ ]:
len(df_url.drop_duplicates())

In [ ]:
s_req = requests.get('https://en.wikipedia.org/Alexander_the_Great')

soup = BeautifulSoup(s_req.text)

ul = soup.find('ul', {'class':'redirectText'})

In [ ]:
if ul is not None:
    print 'works'

In [ ]:
df_battle_all = pd.DataFrame()
df_battle = pd.DataFrame()
def table_scrape(url, name):
#     sleep(1)
    print name
    link = 'https://en.wikipedia.org' + url
    b_req = requests.get(link)

    soup = BeautifulSoup(b_req.text)
    
    bell = soup.find('table', {'class':'infobox vevent'})
    
    if bell is not None:

        details = pd.DataFrame(parse.make2d(bell))
    else:
        return b_req
    

    if 'Belligerents' in str(bell):
    
        clean_rows = details.drop_duplicates(0).set_index(0,drop = True).loc[['Date', 'Location', 'Result'],:]

        clean_col = clean_rows.transpose().reset_index(drop = True)

        df_x_bells = pd.DataFrame()

        html_table = bell.find_all('tr')
        bell_list = []
        bell_y_list = []

        for index, cell in zip(range(0, len(html_table)), html_table):
            if 'Commanders and leaders' in str(cell):
                leaders = html_table[index + 1]
                cells = leaders.find_all('td')
                count = 0
                for cell in cells:
                    count += 1
                    anchors = cell.find_all('a')
                    for anchor in anchors:
                        if 'title' in str(anchor):
                            if 'class="image"' in str(anchor):
                                cl = 'image'
                            elif 'class="thumbborder"' in str(anchor):
                                cl = 'thumb'
                            elif 'cite_note' in str(anchor):
                                cl = 'cite'
                            elif 'disambiguation needed' in str(anchor):
                                cl = 'disam'
                            elif 'cnote_g' in str(anchor):
                                cl = 'cnote'
                            else:
                                title = anchor['title']
                                if any(title == x for x in ['Napoleon I','Alexander III of Macedon']):
                                    print 'napoleon or alex redirect'

                                ref = anchor['href']
                                link = 'https://en.wikipedia.org' + ref

                                if len(ref) <= 10:
                                    t_req = requests.get(link)
                                elif any(ref[8] + ref[9] + ref[10] == x for x in ['pt.', 'tr.', 'ko.', 'ja.', 'th.','da.', 'es.', 'de.', 'it.', 'fr.', 'zh.']):
                                    print 'Not English!'
                                    link = ref
                                    t_req = requests.get(link)
                                elif soup.find('ul', {'class':'redirectText'}) is not None:
                                    print 'REDIRECT'
                                    print title
                                    ul = soup.find('ul', {'class':'redirectText'})
                                    a = ul.a
                                    redirect = a['href']
                                    link = 'https://en.wikipedia.org' + redirect
                                    t_req = requests.get(link)
                                else:
                                    t_req = requests.get(link)

                                t_soup = BeautifulSoup(t_req.text)

                                c_name = t_soup.find('title')
                                c_name = str(c_name)

                                if c_name:
                                    c_name = c_name.replace(' - Wikipedia', '').replace('<title>', '').replace('</title>', '')
                                else:
                                    if count == 1:
                                        bell_list.append(title)
                                    elif count == 2:
                                        bell_y_list.append(title)
                                if 'ex.php?title=' in ref:
                                    ref = ref.replace('ex.php?title=', '')
                                    ref = ref.replace('&action=edit&redlink=1', '')

                                omit_list = ['Capital punishment', 
                                             'Military advisor', 
                                             'Wounded in action', 
                                             'Prisoner of war', 
                                             'Killed in action', 
                                             'Surrender (military)', 
                                             'Surrendered',
                                             'Common military ranks in English']

                                if all(c_name != x for x in omit_list):


                                    if count == 1:

                                        bell_list.append(c_name)
                                    elif count == 2:
                                        bell_y_list.append(c_name)



        for bell in bell_list:
            clean_col['belligerent'] = bell
            df_x_bells = pd.concat([df_x_bells, clean_col])
            df_x_bells = df_x_bells.reset_index(drop = True)


        df_y_bells = pd.DataFrame()
        for bell in bell_y_list:
            clean_col['belligerent'] = bell
            df_y_bells = pd.concat([df_y_bells, clean_col])
            df_y_bells = df_y_bells.reset_index(drop = True)

        df_y_bells = df_y_bells.rename(columns = {0:'Location'})

        if 'Strength' in str(details[0]):
            strength_finder = details[details[0] == 'Strength']


            strength_row = strength_finder.index + 1

            strength = details.loc[strength_row]

            strength_x = strength.iloc[0,0]

            strength_y = strength.iloc[0,1]
        else:
            strength_x = 'unknown'
            strength_y = 'unknown'

        df_x_bells['own'] = strength_x

        df_x_bells['opp'] = strength_y

        df_y_bells['own'] = strength_y

        df_y_bells['opp'] = strength_x

        if 'Casualties and losses' not in str(details[0]):
            df_x_bells['taken'] = np.NaN
            df_x_bells['inflicted'] = np.NaN
            df_y_bells['taken'] = np.NaN
            df_y_bells['inflicted'] = np.NaN

        else:

            casualties_finder = details[details[0] == 'Casualties and losses']

            casualties_row = casualties_finder.index + 1

            casualties = details.loc[casualties_row]

            casualties_x = casualties.iloc[0,0]

            casualties_y = casualties.iloc[0,1]

            df_x_bells['taken'] = casualties_x

            df_x_bells['inflicted'] = casualties_y

            df_y_bells['taken'] = casualties_y

            df_y_bells['inflicted'] = casualties_x



        df_x_bells['Battle'] = name

        df_y_bells['Battle'] = name

        df_x_bells['pos'] = 'L'

        df_y_bells['pos'] = 'R'

        df_battle = pd.concat([df_x_bells, df_y_bells]).reset_index(drop = True)

        df_battle_all = pd.concat([df_battle_all, df_battle]).reset_index(drop = True).drop_duplicates()

        global df_battle_all
    else:
        print 'Filtered: ' + name

In [ ]:
count = 0
for num, rows in {1:df_url[:200],
             2:df_url[200:400], 
             3:df_url[400:600], 
             4:df_url[600:800],
             5:df_url[800:1000],
             6:df_url[1000:1200],
             7:df_url[1200:1400],
             8:df_url[1400:1600],
             9:df_url[1600:1800],
             10:df_url[1800:2000],
             11:df_url[2000:2200],
             12:df_url[2200:2400],
             13:df_url[2400:2600],
             14:df_url[2600:2800],
             15:df_url[2800:3000],
             16:df_url[3000:3200],
             17:df_url[3200:3400],
             18:df_url[3400:]}.iteritems():
    df_battle = pd.DataFrame()
    df_batch = rows
    for index, row in df_batch.iterrows():
        count += 1
        global count
        print str(count) + '/' + str(len(df_url))
        table_scrape(row['url'], row['Battle'])
    df_battle_all.to_csv('rows_{}.csv'.format(num), encoding = 'utf-8')
    print 'NEW FRAME' + str(num)

In [ ]:
row_list = ['rows_1.csv',
 'rows_2.csv',
 'rows_3.csv',
 'rows_4.csv',
 'rows_5.csv',
 'rows_6.csv',
 'rows_7.csv',
 'rows_8.csv',
 'rows_9.csv',
 'rows_10.csv',
 'rows_11.csv',
 'rows_12.csv',
 'rows_13.csv',
 'rows_14.csv',
 'rows_15.csv',
 'rows_16.csv',
 'rows_17.csv',
 'rows_18.csv']

In [ ]:
df_comm = pd.DataFrame()
for sheet in row_list:
    df_rows = pd.read_csv(sheet)
    df_comm = pd.concat([df_comm, df_rows]).reset_index(drop = True)

In [ ]:
df_comm = df_comm.drop('Unnamed: 0', axis = 1)

In [ ]:
len(df_comm)

In [ ]:
len(df_comm)

In [ ]:
df_comm.to_csv('battles_dirty.csv', encoding = 'utf-8')

In [ ]:
df_comm = pd.read_csv('battles_dirty.csv').drop('Unnamed: 0', axis = 1)

In [ ]:
df_comm = df_comm.drop_duplicates()

In [ ]:
df_comm.head()

In [ ]:
len(df_comm)

In [ ]:
df_comm[df_comm.belligerent.str.contains('Nasser')]

In [ ]:
len(df_comm.drop_duplicates())

In [ ]:
df_comm = df_comm.drop_duplicates().reset_index(drop = True)

In [ ]:
df_comm.to_csv('battles_deduped.csv')

In [ ]:
len(df_comm[df_comm.belligerent.str.contains('Napoleon')])

In [ ]:
df_comm[df_comm.belligerent.str.contains('Scipio')]

In [ ]:
df_results = pd.read_csv('result_column.csv')

In [ ]:
df_results[df_results.Battle == 'Battle_of_Camarón']

In [ ]:
df_comm.head()

In [ ]:
df_results.head()

In [ ]:
df_more_results = pd.merge(df_comm, df_results, how = 'outer', on = ['Battle', 'belligerent', 'pos', 'Result'])

In [ ]:
df_more_results = df_more_results[df_more_results.VorD.isnull()]

In [ ]:
df_more_results = df_more_results.drop_duplicates(subset = ['Battle', 'Date', 'VorD'])

In [ ]:
df_more_results.head()

In [ ]:
df_more_results.to_csv('adding_results_10.21.csv', encoding = 'utf-8')

In [ ]:
df_comm[df_comm.Battle == 'Battle_of_Camarón']

In [ ]:
df_comm[df_comm.belligerent == 'Ambroise de Loré']

In [ ]:
df_vl = pd.merge(df_comm, df_results, how = 'outer', on = ['Battle', 'belligerent', 'Result', 'pos'])

In [ ]:
df_vl

In [ ]:
df_vl = df_vl.sort('VorD')

In [ ]:
df_vl[df_vl.belligerent == 'Ambroise de Loré']

In [ ]:
df_vl[df_vl.belligerent == 'Saracens']

In [ ]:
df_vl.to_csv('additional_results.csv')

In [ ]:
df_10_21_fill = df_vl[df_vl.VorD.isnull()]

In [ ]:
df_ar = pd.read_csv('additional_results.csv', encoding = 'utf-8')

In [ ]:
df_strength = df_ar.drop(['Unnamed: 0', 'Location', 'Result', 'inflicted', 'taken', 'opp'], axis = 1)

In [ ]:
df_strength = df_strength.sort('Battle').reset_index(drop = True)

In [ ]:
df_strength_dd = df_strength.drop_duplicates(['own', 'pos'], keep = 'first')

In [ ]:
df_strength_dd.own = df_strength_dd.own.str.replace('\r',' ').str.replace('\n', ' ').replace('\n', ' ')

In [ ]:
df_strength_dd = df_strength_dd.sort('Battle').reset_index(drop = True)

In [ ]:
df_strength_dd

In [ ]:
df_strength_dd.to_csv('strength_entry.csv', encoding = 'utf-8')

In [ ]:
df_strength_new = df_strength.drop_duplicates(['own', 'pos', 'Battle'], keep = 'first')

In [ ]:
df_strength_fill = pd.read_excel('strength_entry.xlsx', encoding = 'utf-8')

In [ ]:
df_battles = pd.read_csv('battles_deduped.csv', encoding = 'utf-8').drop('Unnamed: 0', axis = 1)

In [ ]:
df_battles = df_battles.drop(['taken', 'inflicted'], axis = 1)

In [ ]:
df_strength_fill.Battle = df_strength_fill.Battle.str.replace(u"â€“",u'–')

In [ ]:
df_some_strength = pd.merge(df_battles, df_strength_fill, how = 'outer', on = ['Battle', 'pos']).drop(['own_y', 'belligerent_y', 'Date_y'], axis = 1).rename(columns = {'Date_x':'Date','own_x':'own', 'belligerent_x':'belligerent'})

In [ ]:
df_some_strength[df_some_strength.Battle.str.contains('Battle_of_Jena')]

In [ ]:
df_test = df_some_strength.iloc[:,9:]

In [ ]:
df_str_two = df_some_strength[df_some_strength['Infantry'].isnull() & df_some_strength['Cavalry'].isnull() & df_some_strength['Artillery'].isnull() & df_some_strength['Ships'].isnull() & df_some_strength['Airforce'].isnull() & df_some_strength['Special'].isnull()]

In [ ]:
df_str_two.head()

In [ ]:
df_str_two = df_str_two.drop_duplicates(['Battle', 'Date', 'Location', 'Result', 'pos']).drop('VorD', axis = 1)

In [ ]:
len(df_str_two)

In [ ]:
df_merger = df_strength.drop(['Date', 'belligerent', 'own'], axis = 1)

In [ ]:
df_merger = df_merger.drop_duplicates()

In [ ]:
df_str_two.head()

In [ ]:
df_str_two = pd.merge(df_str_two, df_merger, on = ['Battle', 'pos'], how = 'outer').dropna(subset = ['Date', 'Location', 'Result'])

In [ ]:
df_str_two = df_str_two.dropna(subset = ['VorD'])

In [ ]:
df_str_two.to_csv('strength_entry_two.csv')

In [ ]:
df_str_two

In [ ]:
df_partial = pd.read_excel('PARTIAL_strength_entry_two.xlsx', encoding = 'utf-8')

In [ ]:
df_str_two.head()

In [ ]:
df_str_two = df_str_two.drop(['Infantry', 'Cavalry', 'Artillery', 'Ships', 'Airforce', 'Special'], axis = 1)

In [ ]:
df_partial = df_partial.drop(['Date', 'Location', 'Result', 'belligerent','own', 'Column1'],axis = 1)

In [ ]:
df_partial.columns

In [ ]:
df_str_two.head()

In [ ]:
df_finish = pd.merge(df_str_two, df_partial, on = ['Battle', 'pos', 'VorD'], how = 'outer')

In [ ]:
drop_list = list(df_finish[df_finish['Date'].isnull()].index)

In [ ]:
df_finish = df_finish.drop(drop_list)

In [ ]:
df_finish.to_csv('last_strength_entry.csv', encoding = 'utf-8')

In [ ]:
df_some_strength

In [ ]:
df_some_strength[df_some_strength.Battle.str.contains('Siege of La Rochelle')]

In [ ]:
df_more_strength = pd.read_excel('last_strength_entry.xlsx')

In [ ]:
df_more_strength.head()

In [ ]:
df_some_strength.head()

In [ ]:
df_more_strength = pd.merge(df_more_strength, df_str_two, on = ['Battle', 'pos'], how = 'outer').drop(['own_y', 'Infantry_y', 'Cavalry_y', 'Artillery_y', 'Ships_y', 'Airforce_y', 'Special_y', 'Date_y', 'Location_y', 'Result_y', 'belligerent_y'], axis = 1).dropna(subset = ['Date_x'])

In [ ]:
df_more_strength.columns = df_more_strength.columns.str.replace('_x', '')

In [ ]:
df_more_strength.head()

In [ ]:
df_some_strength.head()

In [ ]:
df_more_strength

In [ ]:
df_fill_two = pd.merge(df_battles, df_more_strength, on = ['Battle', 'pos'], how = 'outer').drop(['Date_y', 
                                                                                    'Location_y', 
                                                                                    'belligerent_y', 
                                                                                    'Result_y', 'own_y', 
                                                                                    'opp_y'], axis = 1)

In [ ]:
df_fill_two.columns = df_fill_two.columns.str.replace('_x', '')

In [ ]:
df_some_strength = df_some_strength[df_some_strength['Infantry'].notnull() | df_some_strength['Cavalry'].notnull() | df_some_strength['Artillery'].notnull() | df_some_strength['Ships'].notnull() | df_some_strength['Airforce'].notnull() | df_some_strength['Special'].notnull()]

In [ ]:
len(df_some_strength)

In [ ]:
df_fill_two = df_fill_two[df_fill_two['Infantry'].notnull() | df_fill_two['Cavalry'].notnull() | df_fill_two['Artillery'].notnull() | df_fill_two['Ships'].notnull() | df_fill_two['Airforce'].notnull() | df_fill_two['Special'].notnull()]

In [ ]:
df_some_strength

In [ ]:
df_fill_two

In [ ]:
df_all_strength = pd.concat([df_fill_two, df_some_strength]).reset_index(drop = True)

In [ ]:
len(df_battles)

In [ ]:
df_char = df_all_strength[df_all_strength.Date.isnull()]

In [ ]:
df_char.to_excel('special_character_fix.xlsx')

In [ ]:
df_all_strength[df_all_strength.Battle.str.contains('Battle_of_Tenn')]

In [ ]:
df_all_strength.to_excel('all_strength_probably.xlsx')

In [ ]:
df_battles[df_battles.Battle.str.contains('dlingen')]

In [ ]:
df_char.Battle

In [6]:
df_strength_all = pd.read_excel('all_strength_probably.xlsx')

In [7]:
df_strength_all = df_strength_all.drop_duplicates()

In [8]:
df_left = df_strength_all[df_strength_all.pos == 'L']

In [9]:
df_left = df_left.drop(['Date', 'Location', 'Result', 'belligerent', 'opp', 'own'], axis = 1)

In [10]:
df_left = df_left.drop_duplicates(subset = ['Battle'])

In [11]:
df_left = df_left.dropna(subset = ['Infantry', 'Cavalry', 'Artillery', 'Ships', 'Airforce', 'Special'], how = 'all')

In [12]:
df_right = df_strength_all[df_strength_all.pos == 'R']

In [13]:
df_right = df_right.drop_duplicates(subset = ['Battle'])

In [14]:
df_right = df_right.drop(['Date', 'Location', 'Result', 'belligerent', 'opp', 'own', 'VorD'], axis = 1)

In [15]:
df_right = df_right.dropna(subset = ['Infantry', 'Cavalry', 'Artillery', 'Ships', 'Airforce', 'Special'], how = 'all')

In [16]:
df_model = pd.merge(df_left, df_right, on = 'Battle', how = 'outer').dropna(subset = ['pos_x', 'pos_y']).reset_index(drop = True)

In [17]:
df_model

,Battle,pos_x,VorD,Infantry_x,Cavalry_x,Artillery_x,Ships_x,Airforce_x,Special_x,pos_y,Infantry_y,Cavalry_y,Artillery_y,Ships_y,Airforce_y,Special_y
0,1898_invasion_of_GuantÃ¡namo_Bay,L,V,923.0,NaN,NaN,6.0,NaN,NaN,R,5000.0,NaN,3.0,2.0,NaN,NaN
1,1948_Arab–Israeli_War,L,V,117500.0,NaN,NaN,NaN,NaN,NaN,R,58800.0,NaN,NaN,NaN,NaN,NaN
2,1982_Lebanon_War,L,I,113000.0,2397.0,NaN,NaN,634.0,NaN,R,37500.0,882.0,1125.0,NaN,450.0,NaN
3,1998_Battle_of_Kilinochchi,L,D,1.0,NaN,NaN,NaN,NaN,NaN,R,1.0,NaN,NaN,NaN,NaN,NaN
4,Abagana_Ambush,L,D,6000.0,NaN,NaN,NaN,NaN,NaN,R,700.0,NaN,NaN,NaN,NaN,NaN
5,Action_at_Bronkhorstspruit,L,D,268.0,NaN,NaN,NaN,NaN,NaN,R,250.0,NaN,NaN,NaN,NaN,NaN
6,Attack_on_Rabaul,L,V,NaN,NaN,NaN,16.0,631.0,NaN,R,NaN,NaN,NaN,21.0,200.0,NaN
7,Attack_on_Sydney_Harbour,L,I,NaN,NaN,NaN,19.0,NaN,NaN,R,NaN,NaN,NaN,8.0,2.0,NaN
8,"Battle of ""The Saw""",L,V,10000.0,2000.0,NaN,NaN,NaN,NaN,R,50000.0,NaN,NaN,NaN,NaN,NaN
9,Battle of Ã‡Ä±ldÄ±r,L,V,1.0,NaN,NaN,NaN,NaN,NaN,R,32000.0,NaN,NaN,NaN,NaN,NaN


In [18]:
df_model.VorD.value_counts()

V    1524
D     775
I     173
L       1
v       1
Name: VorD, dtype: int64

In [19]:
(1524 - 775) / 2

374.5

In [20]:
df_v = df_model[df_model.VorD == 'V']

In [21]:
v_list = list(df_v.index)

In [22]:
ran_list = random.sample(v_list, 374)

In [23]:
df_switch = df_model.iloc[ran_list]

In [24]:
df_switch['Infantry_hold'] = df_switch['Infantry_x']
df_switch['Cavalry_hold'] = df_switch['Cavalry_x']
df_switch['Artillery_hold'] = df_switch['Artillery_x']
df_switch['Ships_hold'] = df_switch['Ships_x']
df_switch['Airforce_hold'] = df_switch['Airforce_x']
df_switch['Special_hold'] = df_switch['Special_x']

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [25]:
df_switch['Infantry_x'] = df_switch['Infantry_y']
df_switch['Cavalry_x'] = df_switch['Cavalry_y']
df_switch['Artillery_x'] = df_switch['Artillery_y']
df_switch['Ships_x'] = df_switch['Ships_y']
df_switch['Airforce_x'] = df_switch['Airforce_y']
df_switch['Special_x'] = df_switch['Special_y']

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [26]:
df_switch['Infantry_y'] = df_switch['Infantry_hold']
df_switch['Cavalry_y'] = df_switch['Cavalry_hold']
df_switch['Artillery_y'] = df_switch['Artillery_hold']
df_switch['Ships_y'] = df_switch['Ships_hold']
df_switch['Airforce_y'] = df_switch['Airforce_hold']
df_switch['Special_y'] = df_switch['Special_hold']

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [27]:
df_switch = df_switch.drop(['Infantry_hold', 'Cavalry_hold', 'Artillery_hold', 'Ships_hold', 'Airforce_hold', 'Special_hold'], axis = 1)

In [28]:
df_switch.VorD = 'D'

In [29]:
df_switch.VorD.value_counts()

D    374
Name: VorD, dtype: int64

In [30]:
df_model = df_model.drop(ran_list)

In [31]:
df_model = pd.concat([df_model, df_switch])

In [32]:
df_model.VorD.value_counts()

V    1150
D    1149
I     173
L       1
v       1
Name: VorD, dtype: int64

In [33]:
df_model.loc[2378, 'Infantry_y'] = 25000

In [34]:
df_model.loc[2363, 'Infantry_x'] = 500

In [35]:
df_model = df_model[df_model.Infantry_x != 1.0]
df_model = df_model[df_model.Infantry_y != 1.0]

In [36]:
df_model = df_model.fillna(0)

In [37]:
df_model['infantry_diff'] = (df_model['Infantry_x'] - df_model['Infantry_y']) / (df_model['Infantry_x'] + df_model['Infantry_y'])
df_model['cavalry_diff'] = (df_model['Cavalry_x'] - df_model['Cavalry_y']) / (df_model['Cavalry_x'] + df_model['Cavalry_y'])
df_model['artillery_diff'] = (df_model['Artillery_x'] - df_model['Artillery_y']) / (df_model['Artillery_x'] + df_model['Artillery_y'])
df_model['ships_diff'] = (df_model['Ships_x'] - df_model['Ships_y']) / (df_model['Ships_x'] + df_model['Ships_y'])
df_model['airforce_diff'] = (df_model['Airforce_x'] - df_model['Airforce_y']) / (df_model['Airforce_x'] + df_model['Airforce_y'])
df_model['special_diff'] = (df_model['Special_x'] - df_model['Special_y']) / (df_model['Special_x'] + df_model['Special_y'])

In [38]:
df_diff = pd.concat([df_model['VorD'], df_model['Battle'], df_model['infantry_diff'],df_model['cavalry_diff'],df_model['artillery_diff'], df_model['ships_diff'],
                    df_model['airforce_diff'], df_model['special_diff']], axis = 1)

In [39]:
df_diff = df_diff[df_diff.VorD != 'I']

In [40]:
df_model[df_model.Battle == 'Siege of Noda']

,Battle,pos_x,VorD,Infantry_x,Cavalry_x,Artillery_x,Ships_x,Airforce_x,Special_x,pos_y,...,Artillery_y,Ships_y,Airforce_y,Special_y,infantry_diff,cavalry_diff,artillery_diff,ships_diff,airforce_diff,special_diff
2363,Siege of Noda,L,V,500.0,0.0,0.0,0.0,0.0,0.0,R,...,0.0,0.0,0.0,0.0,0.818182,NaN,NaN,NaN,NaN,NaN


In [41]:
df_diff.sort_values('infantry_diff', ascending = False)

,VorD,Battle,infantry_diff,cavalry_diff,artillery_diff,ships_diff,airforce_diff,special_diff
1933,D,Battle_of_the_Mississinewa,1.000000,-1.000000,NaN,NaN,NaN,NaN
327,D,Battle of Ikh Bayan,1.000000,-1.000000,NaN,NaN,NaN,NaN
934,D,Battle_of_Brices_Crossroads,1.000000,-0.029412,1.000000,NaN,NaN,NaN
321,V,Battle of Horic,1.000000,-0.818182,NaN,NaN,-1.000000,0.0
644,V,Battle of the Arius,1.000000,-1.000000,NaN,NaN,NaN,NaN
622,V,Battle of Sudomer,1.000000,-1.000000,NaN,NaN,NaN,1.0
2166,D,First_Battle_of_Tucson,1.000000,-1.000000,-1.000000,NaN,NaN,NaN
1639,D,Battle_of_Palo_Duro_Canyon,1.000000,-1.000000,NaN,NaN,NaN,NaN
677,D,Battle of the Ticinus,1.000000,-0.318681,NaN,NaN,NaN,NaN
2045,V,Battle_of_Warsaw_(1705),1.000000,-0.660839,NaN,NaN,NaN,NaN


In [42]:
df_diff = df_diff.drop(174)

In [43]:
df_diff = df_diff.fillna(0)

In [44]:
df_x = pd.DataFrame(df_diff.iloc[:, 2:])

In [45]:
df_x = df_x.drop('artillery_diff', axis = 1)

In [46]:
df_y = pd.DataFrame(df_diff.iloc[:, 0])

In [47]:
df_y.VorD = df_y.VorD.replace('V', 1).replace('D', 0).replace('L', 0).replace('v', 1)

In [48]:
df_y.VorD = df_y.VorD.astype('float')

In [49]:
df_x = df_x.iloc[:, :-1]

In [50]:
lr = LogisticRegression()
lr.fit(df_x, df_y)

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [51]:
lr.score(df_x, df_y)

0.54554655870445345

In [52]:
pd.Series(df_x.columns)

0    infantry_diff
1     cavalry_diff
2       ships_diff
3    airforce_diff
dtype: object

In [53]:
lr.coef_[0]

array([ 0.40970551,  0.09491137,  0.57534553,  0.11246618])

In [54]:
pd.concat([pd.Series(lr.coef_[0]), pd.Series(df_x.columns)], axis = 1)

,0,1
0,0.409706,infantry_diff
1,0.094911,cavalry_diff
2,0.575346,ships_diff
3,0.112466,airforce_diff


In [55]:
df_test = pd.concat([df_y.reset_index(drop = True), pd.Series(lr.predict(df_x))], axis = 1)

In [56]:
df_test.head()

,VorD,0
0,1.0,1.0
1,0.0,1.0
2,0.0,0.0
3,1.0,0.0
4,1.0,0.0


In [57]:
len(df_test)

1976

In [58]:
count = 0
for i, row in df_test.iterrows():
    if row['VorD'] == row[0]:
        count = count + 1

In [59]:
scores = cross_val_score(lr, df_x, df_y.VorD, cv = 10)

In [60]:
scores.mean()

0.53955856608140873

In [61]:
pd.concat([pd.DataFrame(lr.predict_proba(df_x)), df_x.reset_index(drop = True)] ,axis = 1)

,0,1,infantry_diff,cavalry_diff,ships_diff,airforce_diff
0,0.471721,0.528279,0.332955,0.000000,0.000000,0.0
1,0.425333,0.574667,0.791045,0.000000,0.000000,0.0
2,0.502235,0.497765,0.034749,0.000000,0.000000,0.0
3,0.550182,0.449818,-0.666667,1.000000,0.000000,0.0
4,0.509599,0.490401,0.000000,0.000000,-0.026455,0.0
5,0.507473,0.492527,-0.016393,0.000000,0.000000,0.0
6,0.523926,0.476074,-0.073171,0.000000,-0.074074,0.0
7,0.517939,0.482061,-0.118644,0.000000,0.000000,0.0
8,0.546014,0.453986,-0.393939,0.000000,0.000000,0.0
9,0.454706,0.545294,0.500000,0.000000,0.000000,0.0


In [62]:
lr.coef_

array([[ 0.40970551,  0.09491137,  0.57534553,  0.11246618]])

In [63]:
df_test = pd.DataFrame({'infantry_diff':.99,
                       'cavalry_diff':.99,
                       'artillery_diff':.99,
                       'ships_diff':.99,
                       'airforce_diff:':.99}, index = [0])

In [64]:
df_test

,airforce_diff:,artillery_diff,cavalry_diff,infantry_diff,ships_diff
0,0.99,0.99,0.99,0.99,0.99


In [65]:
lr.predict_proba(df_test)

ValueError: X has 5 features per sample; expecting 4

In [66]:
df_model.VorD.value_counts()

D    988
V    977
I    128
0     10
L      1
v      1
Name: VorD, dtype: int64

In [67]:
1524 + 775

2299

In [68]:
2300 /2 - 775

375.0

In [69]:
rf = RandomForestClassifier()
rf.fit(df_x, df_y)

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [70]:
rf.score(df_x, df_y)

0.83097165991902833

In [71]:
scores = cross_val_score(rf, df_x, df_y.VorD, cv = 5)

In [72]:
scores

array([ 0.48484848,  0.46969697,  0.5443038 ,  0.49620253,  0.53045685])

In [73]:
rf.feature_importances_

array([ 0.76163327,  0.10735528,  0.10296242,  0.02804903])

In [74]:
pd.concat([pd.DataFrame(rf.predict_proba(df_x)), df_x.reset_index(drop = True)], axis = 1)

,0,1,infantry_diff,cavalry_diff,ships_diff,airforce_diff
0,0.193643,0.806357,0.332955,0.000000,0.000000,0.0
1,0.766667,0.233333,0.791045,0.000000,0.000000,0.0
2,0.800000,0.200000,0.034749,0.000000,0.000000,0.0
3,0.100000,0.900000,-0.666667,1.000000,0.000000,0.0
4,0.000000,1.000000,0.000000,0.000000,-0.026455,0.0
5,0.100000,0.900000,-0.016393,0.000000,0.000000,0.0
6,0.000000,1.000000,-0.073171,0.000000,-0.074074,0.0
7,0.400000,0.600000,-0.118644,0.000000,0.000000,0.0
8,1.000000,0.000000,-0.393939,0.000000,0.000000,0.0
9,0.425122,0.574878,0.500000,0.000000,0.000000,0.0


In [75]:
df_strength_all.loc[60, 'VorD'] = 'V'
df_strength_all.loc[61, 'VorD'] = 'V'

In [76]:
df_strength_all.loc[3479:3485, 'VorD'] = 'V'

In [77]:
df_strength_all.loc[3485, 'VorD'] = 'D'

In [78]:
df_strength_all.loc[3802:3803, 'VorD'] = 'V'
df_strength_all.loc[3804:3807, 'VorD'] = 'D'

In [79]:
df_strength_all.loc[4142:4143, 'VorD'] = 'D'

In [80]:
df_strength_all.VorD = df_strength_all.VorD.str.replace('L', 'D')

In [81]:
df_strength_all.VorD = df_strength_all.VorD.str.replace('v', 'V')

In [82]:
df_strength_all.VorD = df_strength_all.VorD.str.replace('W', 'V')

In [83]:
null_fill = df_strength_all[df_strength_all.VorD.isnull()]

In [84]:
null_fill

,Battle,Date,Location,Result,belligerent,opp,own,pos,VorD,Infantry,Cavalry,Artillery,Ships,Airforce,Special
4981,Battle_of_Gadebusch,9 December 1712 (O.S.)\r\r\r\n20 December 1712...,"Wakenstädt near Gadebusch, 35 km southern of L...",Swedish victory,Sweden,"17,000:[b]\r\r\r\n8,300 Danish foot,\r\r\r\n4,...","12,500:[a]\r\r\r\n6,500 foot,\r\r\r\n6,000 hor...",L,NaN,6500.0,6000.0,25.0,NaN,NaN,NaN
4982,Battle_of_Gadebusch,9 December 1712 (O.S.)\r\r\r\n20 December 1712...,"Wakenstädt near Gadebusch, 35 km southern of L...",Swedish victory,Magnus Stenbock,"17,000:[b]\r\r\r\n8,300 Danish foot,\r\r\r\n4,...","12,500:[a]\r\r\r\n6,500 foot,\r\r\r\n6,000 hor...",L,NaN,6500.0,6000.0,25.0,NaN,NaN,NaN
5100,Battle_of_Goose_Green,28–29 May 1982,"Goose Green and Darwin, Falkland Islands",British victory,Ítalo Piaggi,690[6],684[4]–871 army\r\r\r\n202 airforce\r\r\r\n10 ...,L,NaN,990.0,NaN,NaN,NaN,NaN,NaN
5101,Battle_of_Goose_Green,28–29 May 1982,"Goose Green and Darwin, Falkland Islands",British victory,Wing commander (rank),690[6],684[4]–871 army\r\r\r\n202 airforce\r\r\r\n10 ...,L,NaN,990.0,NaN,NaN,NaN,NaN,NaN
6236,Battle_of_Meissen,4 December 1759,"Meissen, Saxony, present-day Germany",Austrian victory,Habsburg Monarchy,"26,000","21,000",L,NaN,21000.0,NaN,NaN,NaN,NaN,NaN
6237,Battle_of_Meissen,4 December 1759,"Meissen, Saxony, present-day Germany",Austrian victory,Count Leopold Joseph von Daun,"26,000","21,000",L,NaN,21000.0,NaN,NaN,NaN,NaN,NaN
6307,Battle_of_Mobile_Bay,"August 2, 1864 (1864-08-02)–August 23, 1864 (1...","Mobile Bay, Alabama",Union victory,David Farragut,"3 gunboats\r\r\r\n1 ironclad\r\r\r\n1,500 men",12 wooden ships\r\r\r\n2 gunboats\r\r\r\n4 iro...,L,NaN,5500.0,NaN,NaN,18.0,NaN,NaN
6308,Battle_of_Mobile_Bay,"August 2, 1864 (1864-08-02)–August 23, 1864 (1...","Mobile Bay, Alabama",Union victory,Union Navy,"3 gunboats\r\r\r\n1 ironclad\r\r\r\n1,500 men",12 wooden ships\r\r\r\n2 gunboats\r\r\r\n4 iro...,L,NaN,5500.0,NaN,NaN,18.0,NaN,NaN
6309,Battle_of_Mobile_Bay,"August 2, 1864 (1864-08-02)–August 23, 1864 (1...","Mobile Bay, Alabama",Union victory,Gordon Granger,"3 gunboats\r\r\r\n1 ironclad\r\r\r\n1,500 men",12 wooden ships\r\r\r\n2 gunboats\r\r\r\n4 iro...,L,NaN,5500.0,NaN,NaN,18.0,NaN,NaN
6310,Battle_of_Mobile_Bay,"August 2, 1864 (1864-08-02)–August 23, 1864 (1...","Mobile Bay, Alabama",Union victory,Union Army,"3 gunboats\r\r\r\n1 ironclad\r\r\r\n1,500 men",12 wooden ships\r\r\r\n2 gunboats\r\r\r\n4 iro...,L,NaN,5500.0,NaN,NaN,18.0,NaN,NaN


In [85]:
null_fill['VorD'] = ['V', 'D', 'D', 'V', 'V', 'D', 'V', 'V', 'V', 'V', 'D', 'D', 'D', 'D', 'V', 'V', 'V', 'V', 'V',
                    'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D','D', 'D', 'D', 'D', 'V', 'V', 'V', 'V', 'V', 'D', 'D', 'D', 'D',
                    'V', 'V', 'D', 'D', 'D', 'D', np.NaN, 'D', np.NaN]

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [86]:
df_strength_all.VorD.loc[null_fill.index] = null_fill['VorD']

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [87]:
df_strength_all = df_strength_all[df_strength_all.belligerent != 'Sweden']

In [88]:
import re

In [89]:
year_list = []
for i, row in df_strength_all.iterrows():
    d = row['Date']
    if isinstance(d, str):
        if re.search(r'[12]\d{3}', d):
            year = re.search(r'[12]\d{3}', d).group(0)
            year = int(year)
            if 'BC' in d:
                year = year * -1
            year_list.append({'Year':year, 'Index':i})
        elif re.search(r'[12]\d{2}', d):
            year = re.search(r'[12]\d{2}', d).group(0)
            year = int(year)
            if 'BC' in d:
                year = year * -1
            year_list.append({'Year':year, 'Index':i})
    

In [90]:
df_years = pd.DataFrame(year_list).set_index('Index', drop = True)

In [91]:
df_strength_all['year'] = pd.Series()

In [92]:
df_strength_all['year'].loc[df_years.index] = df_years.Year

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [93]:
df_strength_all.to_csv('current_run.csv', encoding = 'utf-8')

In [94]:
df_strength_all = pd.read_csv('current_run.csv', encoding = 'utf-8').drop('Unnamed: 0', axis = 1)

In [95]:
df_run = df_strength_all[df_strength_all.VorD.notnull()]

In [96]:
df_run.Battle = df_run.Battle.str.replace('_'," ")

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [97]:
df_run

,Battle,Date,Location,Result,belligerent,opp,own,pos,VorD,Infantry,Cavalry,Artillery,Ships,Airforce,Special,year
0,1898 invasion of GuantÃ¡namo Bay,NaN,NaN,NaN,NaN,NaN,NaN,R,D,5000.0,NaN,3.0,2.0,NaN,NaN,NaN
1,1898 invasion of GuantÃ¡namo Bay,NaN,NaN,NaN,NaN,NaN,NaN,R,D,5000.0,NaN,7.0,2.0,NaN,NaN,NaN
2,1898 invasion of GuantÃ¡namo Bay,NaN,NaN,NaN,NaN,NaN,NaN,L,V,923.0,NaN,NaN,6.0,NaN,NaN,NaN
3,1948 Arab–Israeli War,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...","Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,David Ben-Gurion,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,V,117500.0,NaN,NaN,NaN,NaN,NaN,1948.0
4,1948 Arab–Israeli War,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...","Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,Yisrael Galili,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,V,117500.0,NaN,NaN,NaN,NaN,NaN,1948.0
5,1948 Arab–Israeli War,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...","Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,Yaakov Dori,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,V,117500.0,NaN,NaN,NaN,NaN,NaN,1948.0
6,1948 Arab–Israeli War,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...","Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,Yigael Yadin,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,V,117500.0,NaN,NaN,NaN,NaN,NaN,1948.0
7,1948 Arab–Israeli War,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...","Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,Mickey Marcus,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,V,117500.0,NaN,NaN,NaN,NaN,NaN,1948.0
8,1948 Arab–Israeli War,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...","Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,Yigal Allon,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,V,117500.0,NaN,NaN,NaN,NaN,NaN,1948.0
9,1948 Arab–Israeli War,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...","Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,Yitzhak Rabin,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,V,117500.0,NaN,NaN,NaN,NaN,NaN,1948.0


In [98]:
arras_1 = {'Battle':['Battle of Arras', 'Battle of Arras'], 
           'Date':['21 May 1940', '21 May 1940'],
           'Location':['Arras', 'Arras'], 
           'Result':['German victory', 'German victory'] ,
           'belligerent':['Erwin Rommel', 'Harold Franklyn'],
           'opp':['manual', 'manual'],
           'own':['manual', 'manual'], 
           'pos':['R', 'L'],
           'VorD':['V', 'D'],
           'Infantry':[7500.0, 2000.0],
           'Cavalry':[225.0, 74.0],
           'Artillery':[np.NaN, np.NaN],
           'Ships': [np.NaN, np.NaN],
           'Airforce': [np.NaN, np.NaN],
           'Special': [np.NaN, np.NaN],
           'year':[1940, 1940]}

In [99]:
df_run = pd.concat([df_run,pd.DataFrame(arras_1, index = [10515,10516])])

In [100]:
df_run

,Airforce,Artillery,Battle,Cavalry,Date,Infantry,Location,Result,Ships,Special,VorD,belligerent,opp,own,pos,year
0,NaN,3.0,1898 invasion of GuantÃ¡namo Bay,NaN,NaN,5000.0,NaN,NaN,2.0,NaN,D,NaN,NaN,NaN,R,NaN
1,NaN,7.0,1898 invasion of GuantÃ¡namo Bay,NaN,NaN,5000.0,NaN,NaN,2.0,NaN,D,NaN,NaN,NaN,R,NaN
2,NaN,NaN,1898 invasion of GuantÃ¡namo Bay,NaN,NaN,923.0,NaN,NaN,6.0,NaN,V,NaN,NaN,NaN,L,NaN
3,NaN,NaN,1948 Arab–Israeli War,NaN,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...",117500.0,"Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,NaN,NaN,V,David Ben-Gurion,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,1948.0
4,NaN,NaN,1948 Arab–Israeli War,NaN,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...",117500.0,"Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,NaN,NaN,V,Yisrael Galili,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,1948.0
5,NaN,NaN,1948 Arab–Israeli War,NaN,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...",117500.0,"Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,NaN,NaN,V,Yaakov Dori,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,1948.0
6,NaN,NaN,1948 Arab–Israeli War,NaN,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...",117500.0,"Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,NaN,NaN,V,Yigael Yadin,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,1948.0
7,NaN,NaN,1948 Arab–Israeli War,NaN,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...",117500.0,"Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,NaN,NaN,V,Mickey Marcus,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,1948.0
8,NaN,NaN,1948 Arab–Israeli War,NaN,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...",117500.0,"Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,NaN,NaN,V,Yigal Allon,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,1948.0
9,NaN,NaN,1948 Arab–Israeli War,NaN,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...",117500.0,"Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,NaN,NaN,V,Yitzhak Rabin,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,1948.0


In [101]:
len(arras_1)

16

In [102]:
df_lookup = df_strength_all.drop_duplicates(subset = ['Battle', 'pos'])

In [103]:
df_lookup.Battle = df_lookup.Battle.str.replace('_', ' ')

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [104]:
df_lookup = pd.concat([df_lookup, pd.DataFrame(arras_1, index = [10515,10516])])

In [105]:
df_lookup[df_lookup['Battle'] == 'Six Day War']

,Airforce,Artillery,Battle,Cavalry,Date,Infantry,Location,Result,Ships,Special,VorD,belligerent,opp,own,pos,year
10330,957.0,NaN,Six Day War,2504.0,"June 5–10, 1967 (6 days)",547000.0,Middle East,Decisive Israeli victory,NaN,NaN,D,Gamal Abdel Nasser,"50,000 troops\r\r\r\n214,000 reserves\r\r\r\n3...","Egypt: 240,000\r\r\r\nSyria, Jordan, and Iraq:...",R,1967.0
10342,300.0,NaN,Six Day War,800.0,"June 5–10, 1967 (6 days)",264000.0,Middle East,Decisive Israeli victory,NaN,NaN,V,Levi Eshkol,"Egypt: 240,000\r\r\r\nSyria, Jordan, and Iraq:...","50,000 troops\r\r\r\n214,000 reserves\r\r\r\n3...",L,1967.0


In [106]:
war_list = []
battle_list = []
def commander_war(general):
#     print general
    war = 0
    global df_battles
    df_dbg = df_run[df_run.belligerent == general]
    df_battles = pd.DataFrame()
    for battle in df_dbg.Battle.unique():
        global row
        row = df_dbg[df_dbg.Battle == battle].reset_index(drop = True)
        
        battle = row.loc[0,'Battle']

        df_str = df_lookup[df_lookup.Battle == battle].reset_index(drop = True)
        
        outcome = pd.DataFrame(row.loc[:,'VorD']).iloc[0,0]
        
        
        if row.loc[0, 'pos'] == 'L':
            opp_row = df_str[df_str.pos == 'R']
        if row.loc[0, 'pos'] == 'R':
            opp_row = df_str[df_str.pos == 'L']
            
       
            
        
        year = row.loc[0, 'year']

        own = row.drop(['Battle', 'Date', 'Location', 'Result', 'belligerent', 'opp', 'own', 'pos', 'VorD', 'year'], axis = 1)

        opp = opp_row.drop(['Battle', 'Date', 'Location', 'Result', 'belligerent', 'opp', 'own', 'pos', 'VorD', 'year'], axis = 1)
        
        opp = opp.rename(columns = {'Infantry':'infantry_opp', 
                                    'Cavalry':'cavalry_opp', 
                                    'Artillery':'artillery_opp', 
                                    'Ships':'ships_opp', 
                                    'Airforce':'airforce_opp',
                                    'Special':'special_opp'})

        opp = opp.reset_index(drop= True)

        pred_row = pd.concat([own, opp], axis = 1)
        
        pred_diff = pd.DataFrame()
        
        pred_diff['infantry_diff'] = (pred_row['Infantry'] - pred_row['infantry_opp']) / (pred_row['Infantry'] + pred_row['infantry_opp'])
        pred_diff['cavalry_dfff'] = (pred_row['Cavalry'] - pred_row['cavalry_opp']) / (pred_row['Cavalry'] + pred_row['cavalry_opp'])
        pred_diff['artillery_diff'] = (pred_row['Artillery'] - pred_row['artillery_opp']) / (pred_row['Artillery'] + pred_row['artillery_opp'])
        pred_diff['ships_diff'] = (pred_row['Ships'] - pred_row['ships_opp']) / (pred_row['Ships'] + pred_row['ships_opp'])
        pred_diff['airforce_diff'] = (pred_row['Airforce'] - pred_row['airforce_opp']) / (pred_row['Airforce'] + pred_row['airforce_opp'])
        pred_diff['special_diff'] = (pred_row['Special'] - pred_row['special_opp']) / (pred_row['Special'] + pred_row['special_opp'])

#         pred_diff = pred_diff.drop(['Infantry', 'Cavalry', 'Artillery', 'Ships', 'Airforce', 'Special', 'infantry_opp', 'cavalry_opp', 'artillery_opp', 'ships_opp', 'airforce_opp', 'special_opp'], axis = 1)

        pred_diff = pred_diff.fillna(0)

        pred_diff = pred_diff.drop(['artillery_diff', 'special_diff'], axis = 1)

        df_pred = pd.DataFrame(lr.predict_proba(pred_diff))
        
#         if row.loc[0, 'Infantry'] == 1.0:
        

        if len(df_str) == 1:
            if row.loc[0, 'VorD'] == 'V':
                value = 0.5
#                 print 'UNKNOWN'
            elif row.loc[0, 'VorD'] == 'D':
                value = -.5
#                 print 'UNKNOWN'
            elif row.loc[0, 'VorD'] == 'I':
                value = 0
#                 print 'UNKNOWN'
        else:
            if df_str.Infantry.loc[0] == 1.0 or df_str.Infantry.loc[1] == 1.0:
                if row.loc[0, 'VorD'] == 'V':
                    value = 0.5
#                     print 'UNKNOWN'
                elif row.loc[0, 'VorD'] == 'D':
                    value = -.5
#                     print 'UNKNOWN'
                elif row.loc[0, 'VorD'] == 'I':
                    value = 0
#                     print 'UNKNOWN'
            else:
                if row.loc[0, 'VorD'] == 'V':
                    value = df_pred.iloc[0,0]
                elif row.loc[0, 'VorD'] == 'D':
                    value = 0 - df_pred.iloc[0,1]
                elif row.loc[0, 'VorD'] == 'I':
                    value = .5 - df_pred.iloc[0,1]
        
        

#         print u' '.join(battle).encode('utf-8').strip(' ') +': ' + str(value).encode('utf-8')

        
        
        
        war = war + value
        battle_list.append({'General':general,
                            'Battle': battle,
                            'Value': value,
                            'Year': year,
                            'Outcome': outcome})
        df_battles = pd.DataFrame(battle_list)
        
#     print 'Total: ' + str(war)
#     print u' '.join(general).encode('utf-8').strip(' ') + ': ' + str(war)
#     if len(df_dbg) > 4:
#         war_list.append({'General':general,
#                      'WAR':war})
    war_list.append({'General': general,
                    'WAR': war,
                    'Battles': len(df_dbg)})
    print(war)
    

In [107]:
war_list = []
count = 0
for general in df_run.belligerent.unique():
    count += 1
    print(str(general) + ': ' + str(count) + '/' + str(len(df_run)))
    commander_war(general)

nan: 1/10515
0
David Ben-Gurion: 2/10515
0.986016936844
Yisrael Galili: 3/10515
0.471721466161
Yaakov Dori: 4/10515
0.471721466161
Yigael Yadin: 5/10515
0.471721466161
Mickey Marcus: 6/10515
0.471721466161
Yigal Allon: 7/10515
0.471721466161
Yitzhak Rabin: 8/10515
1.03976631482
David Shaltiel: 9/10515
0.471721466161
Moshe Dayan: 10/10515
2.11324527998
Shimon Avidan: 11/10515
0.471721466161
Moshe Carmel: 12/10515
0.471721466161
Yitzhak Sadeh: 13/10515
0.471721466161
Abdul Rahman Hassan Azzam: 14/10515
-0.460186422825
Farouk of Egypt: 15/10515
-0.460186422825
Abdullah I of Jordan: 16/10515
-0.460186422825
Muzahim al-Pachachi: 17/10515
-0.460186422825
Husni al-Za'im: 18/10515
-0.460186422825
Amin al-Husseini: 19/10515
-0.460186422825
Ahmed Ali al-Mwawi: 20/10515
-0.460186422825
Mohammed Naguib: 21/10515
-0.460186422825
John Bagot Glubb: 22/10515
-0.460186422825
Habis al-Majali: 23/10515
-0.460186422825
Hasan Salama: 24/10515
-0.460186422825
Arab Liberation Army: 25/10515
-0.460186422825
F

-0.5
Telets of Bulgaria: 202/10515
-0.5
William Eure, 1st Baron Eure: 203/10515
-0.53008470727
Brian Layton: 204/10515
-0.53008470727
James Hamilton, Duke of Châtellerault: 205/10515
0.0201378971631
Archibald Douglas, 6th Earl of Angus: 206/10515
0.0201378971631
Sophron of Ephesus: 207/10515
-0.5
Ptolemy Andromachou: 208/10515
-0.5
Antigonus II Gonatas: 209/10515
0.5
Oda Nobunaga: 210/10515
4.29314369333
Tokugawa Ieyasu: 211/10515
1.39299217061
Inaba Yoshimichi: 212/10515
0.483534003218
Azai Nagamasa: 213/10515
-0.832035073868
Asakura Yoshikage: 214/10515
-1.35682614218
Timur: 215/10515
0.480766586629
Shahrukh Mirza: 216/10515
0.480766586629
Khalil Sultan: 217/10515
0.480766586629
Miran Shah: 218/10515
0.480766586629
Abu Bakr (Timurid): 219/10515
0.480766586629
Sultan Huseyn: 220/10515
0.480766586629
Pir Muhammad ibn Jahangir: 221/10515
0.480766586629
Taj al-Din Shah-i Shahan Abu'l Fath: 222/10515
0.480766586629
Bayezid I: 223/10515
0.000824814520108
Süleyman Çelebi: 224/10515
-0.46920

-0.0213705286756
John Neville, 1st Marquess of Montagu: 401/10515
1.0124115253
Henry Holland, 3rd Duke of Exeter: 402/10515
-1.02666461104
John de Vere, 13th Earl of Oxford: 403/10515
0.487645596285
Edward IV of England: 404/10515
3.06749520967
Richard III of England: 405/10515
0.538482550939
William Hastings, 1st Baron Hastings: 406/10515
1.05316895004
Suleiman II (Rûm): 407/10515
-0.526525594329
David Soslan: 408/10515
0.538066020894
Zakarids-Mkhargrzeli: 409/10515
0.538066020894
Shalva of Akhaltsikhe: 410/10515
0.538066020894
Nouri al-Maliki: 411/10515
-0.025356822928
Ali Ghaidan Majid: 412/10515
-0.025356822928
Mohan al-Furayji: 413/10515
-0.025356822928
Fadhil Jalil al-Barwari: 414/10515
-0.025356822928
Muqtada al-Sadr: 415/10515
0.536900681156
Shiek Ali al-Sauidi: 416/10515
0.0369006811559
Akram al-Kabi: 417/10515
0.0369006811559
Yusuf al-Mosawi: 418/10515
0.0369006811559
John Stewart, Earl of Buchan: 419/10515
-0.597317031099
Gilbert Motier de La Fayette: 420/10515
0.45080169221

0.437954449788
Konrad von Landau: 591/10515
-0.426578365403
Boudica: 592/10515
-0.14262814234
Quintus Petillius Cerialis: 593/10515
-0.43315214154
Murong Bao: 594/10515
-0.554774283302
Murong Nong: 595/10515
-0.554774283302
Murong Lin: 596/10515
-0.554774283302
Murong De: 597/10515
-0.554774283302
Emperor Daowu of Northern Wei: 598/10515
0.566193462261
Gaius Terentius Varro: 599/10515
-0.523122420192
Lucius Aemilius Paullus (consul 219 BC): 600/10515
-0.523122420192
Maharbal: 601/10515
0.534672487885
Mago Barca: 602/10515
0.0560517892361
Gisgo: 603/10515
0.534672487885
Hanno, son of Bomilcar: 604/10515
0.534672487885
Hasdrubal (quartermaster): 605/10515
0.534672487885
Marcus Claudius Marcellus: 606/10515
0.494414174376
Lucius Manlius Vulso Longus: 607/10515
0.510023789238
Hanno the Great: 608/10515
-0.998435836382
Appius Claudius Pulcher (consul 212 BC): 609/10515
-0.523923757621
Francesco I Sforza: 610/10515
1.45178619808
Micheletto Attendolo: 611/10515
0.0451652519594
William Kirkcal

-0.5
Landulf IV of Benevento: 783/10515
-0.5
Abu al-Qasim (Kalbid): 784/10515
0.5
Abu al-Qasim (Kalbite) — Wikipédia: 785/10515
0.5
Emirate of Sicily: 786/10515
0.5
Owain Gwynedd: 787/10515
0.5
Cadwaladr ap Gruffydd: 788/10515
0.5
Gruffydd ap Rhys: 789/10515
0.5
Robert fitz Martin: 790/10515
-0.5
Maurice FitzGerald, Lord of Lanstephan: 791/10515
-0.5
Avidius Cassius: 792/10515
0.5
Vologases IV: 793/10515
-0.5
Merena: 794/10515
0
Artaxerxes II of Persia: 795/10515
-0.0462522153735
Gobrias: 796/10515
-0.0462522153735
Tissaphernes: 797/10515
-0.0462522153735
Cyrus the Younger: 798/10515
0.0577157140745
Clearchus of Sparta: 799/10515
-0.431292501142
Ariaeus: 800/10515
0.0577157140745
Lamba Doria: 801/10515
0.527560079662
Andrea Dandolo (admiral): 802/10515
-0.515993282996
Titus Quinctius Flamininus: 803/10515
0.990714964952
Ubba: 804/10515
-0.5
Odda, Ealdorman of Devon: 805/10515
0.5
Alcibiades: 806/10515
0.500595744881
Theramenes: 807/10515
0.500595744881
Hermocrates: 808/10515
0.01099178

...

1.04303256176
Liu Yalou: 5853/10515
0.493503850842
Fan Hanjie: 5854/10515
-0.481922067758
Liu Yuzhang: 5855/10515
-0.481922067758
Ahmad Zeidan: 5856/10515
-0.494205608839
Salah al-Qadhi: 5857/10515
-0.494205608839
Juwad Shitnah: 5858/10515
-0.494205608839
Hossein Kharrazi: 5859/10515
0.505794391161
Jaber Al-Khaled Al-Sabah: 5860/10515
0.492435265578
Henri Mathias Berthelot: 5861/10515
0.459327275759
Phraya Bhijai Janriddhi: 5862/10515
0.459327275759
พระยาเทพหัสดิน (ผาด เทพหัสดิน ณ อยุธยา) - วิกิพีเดีย: 5863/10515
0.459327275759
C. Odumegwu Ojukwu: 5864/10515
-1.0
Victor Banjo: 5865/10515
-0.5
Albert Okonkwo: 5866/10515
-0.5
Festus Akagha: 5867/10515
-1.0
Humphrey Chukwuka: 5868/10515
-0.5
Benjamin Adekunle: 5869/10515
0.464882017875
Ma Zhanshan: 5870/10515
-0.549198307788
Feng Zhanhai: 5871/10515
-0.549198307788
Shigeru Honjō: 5872/10515
0.56064724079
Jirō Minami: 5873/10515
0.56064724079
Juan José Latorre: 5874/10515
0.487793393987
Galvarino Riveros: 5875/10515
0.434259265842
Shizuo S

0.386718336945
Willem Schoutens: 6046/10515
-0.375783308167
Hans Kyff: 6047/10515
-0.375783308167
Fath-Ali Shah Qajar: 6048/10515
-0.505585825624
Aleksey Petrovich Yermolov: 6049/10515
0.517168606883
Valerian Madatov: 6050/10515
0.517168606883
Baltacı Mehmet Pasha: 6051/10515
0.43990977209
Devlet II Giray: 6052/10515
0.43990977209
Carl Ewald von Rönne: 6053/10515
-0.428521621525
Ivan Skoropadsky: 6054/10515
-0.428521621525
Dimitrie Cantemir: 6055/10515
-0.428521621525
Dietrich von Falkenberg: 6056/10515
-0.411348824368
Christian Wilhelm von Brandenburg: 6057/10515
-0.411348824368
Robert George Broadwood: 6058/10515
-0.505585825624
Rickard de Bermingham: 6059/10515
0.5
William Liath de Burgh: 6060/10515
0.5
Fedlim Ó Conchobair: 6061/10515
-0.5
Tadhg Ó Cellaigh: 6062/10515
-0.5
List of monarchs of Thomond: 6063/10515
-0.5
Ualgarg Mór Ó Ruairc: 6064/10515
-0.5
Richard F. Natonski: 6065/10515
0.449395793896
Keith J. Stalder: 6066/10515
0.449395793896
Abu Musab al-Zarqawi: 6067/10515
-0.437

0.4573805647
Yang Ang: 6237/10515
0.422618097196
Wei Kang: 6238/10515
-0.411348824368
Kim Si-min: 6239/10515
0.566590727251
Gwak Jae-u: 6240/10515
0.566590727251
Kim Seong-il: 6241/10515
0.566590727251
Yi Gwang-ak: 6242/10515
0.566590727251
Seong Su-gyeong: 6243/10515
0.566590727251
Choi Dak-ryang: 6244/10515
0.566590727251
Shim Dae-seung: 6245/10515
0.566590727251
Kim Jun-min: 6246/10515
0.566590727251
Jeong Gi-ryong: 6247/10515
0.0472082812529
Im Gye-yeong: 6248/10515
0.566590727251
Choi Gyeong-hoe: 6249/10515
0.566590727251
Yu Sung-in: 6250/10515
0.566590727251
Hwang Jin: 6251/10515
-0.470387327995
Kim Cheon-il: 6252/10515
-0.470387327995
Ivan the Terrible: 6253/10515
0.5
Alexander Gorbatyi-Shuisky: 6254/10515
0.5
Andrey Kurbsky: 6255/10515
0.5
Shahghali: 6256/10515
0.5
Yadegar Mokhammad of Kazan: 6257/10515
-0.5
Yapancha beg: 6258/10515
-0.5
Zaynash morza: 6259/10515
-0.5
Qolsharif: 6260/10515
-0.5
Afonso Pereira de Lacerda: 6261/10515
0.5
Diogo de Melo Coutinho: 6262/10515
0.5
Gab

0.499370942885
Fenwick Williams: 6431/10515
-0.487784072895
Mehmet Vasıf Pasha Gürcü: 6432/10515
-0.487784072895
George White (British Army officer): 6433/10515
0.531740692384
Ivan Fedyuninsky: 6434/10515
0.493107831942
Mikhail Khozin: 6435/10515
0.493107831942
Archibald H. Gillespie: 6436/10515
-0.441706539224
Begum Hazrat Mahal: 6437/10515
0.446698466934
Birjis Qadr: 6438/10515
0.446698466934
John Eardley Inglis: 6439/10515
-0.435271151304
Henry Havelock: 6440/10515
-0.435271151304
Colin Campbell, 1st Baron Clyde: 6441/10515
-0.435271151304
James George Smith Neill: 6442/10515
-0.435271151304
Robert Baden-Powell, 1st Baron Baden-Powell: 6443/10515
-0.424701266092
Bryan Mahon: 6444/10515
-0.424701266092
Joseph Anthelme Fournier: 6445/10515
-0.5
Hans von Zwehl: 6446/10515
0.5
Diwan Mulraj Chopra: 6447/10515
0.5
Ferdinand II, Holy Roman Emperor: 6448/10515
-0.554512486831
Tiryaki Hasan Pasha: 6449/10515
0.565933126744
Octavius Ignatius, Prince of Arenberg and Barbançon: 6450/10515
-0.40

0.522324168082
Jan Kiliński: 6611/10515
0.522324168082
James W. Forsyth: 6612/10515
0.447615567499
Spotted Elk: 6613/10515
-0.436183285838
Anwar Sadat: 6614/10515
-0.547727188297
Yusuf Shakkour: 6615/10515
-0.547727188297
Mohamed Abdel Ghani el-Gamasy: 6616/10515
-0.547727188297
Ali Aslan: 6617/10515
-0.547727188297
Golda Meir: 6618/10515
0.559183494476
Yitzhak Hofi: 6619/10515
0.559183494476
Harold Franklyn: 6620/10515
-0.423527703203


In [108]:
pred_row

NameError: name 'pred_row' is not defined

In [ ]:
row.loc[0, 'Battle']

In [ ]:
df_run[df_run.belligerent == 'Alexane']

In [ ]:
df_totalwar = pd.DataFrame(war_list).sort_values('WAR', ascending = False).reset_index(drop = True)

In [ ]:
df_totalwar[df_totalwar.General == 'Alexander the Great']

In [ ]:
df_totalwar[df_totalwar.General == 'Ariel Sharon']

In [ ]:
df_totalwar

In [ ]:
std = np.std(df_totalwar.WAR)

In [ ]:
tw_mean = df_totalwar.WAR.mean()

In [ ]:
(16.678901 - tw_mean) / std

In [ ]:
(7.428568 - tw_mean) / std

In [ ]:
def graph(commander):
    battle_list = []
    global df_battles
    df_battles = pd.DataFrame()
    commander_war(commander)

    df_battles.Year = df_battles.Year.fillna(-5000)
    df_battles.Year = df_battles.Year.astype(float, errors = 'ignore')
    df_battles = df_battles.sort_values('Year')
    wl = pd.DataFrame(df_battles.Outcome.value_counts())
    
    if 'V' in wl.index:
        wins = wl.loc['V', 'Outcome']
    else:
        wins = 0
    if 'D' in wl.index:
        losses = wl.loc['D', 'Outcome']
    else:
        losses = 0
    if 'I' in wl.index:
        draws = wl.loc['I', 'Outcome']
    else:
        draws = 0
    df_battles = df_battles.reset_index(drop = True)
    for i, row in df_battles.iterrows():
        df_battles.ix[i, 'cumulative'] = sum(df_battles.Value[0:i+1])

    plt.figure(figsize = (12,9))
    plt.style.use('fivethirtyeight')
    plt.bar(df_battles.index, 
            df_battles['Value'], 
            label = 'Battle Value', 
            color = df_battles.Outcome.map({'V':'blue', 'I':'orange', 'D':'red'}), 
            alpha = .5, 
            zorder = 2)
    if max(df_battles.cumulative) < 1 and min(df_battles.cumulative) > 1:
        plt.ylim(-1,1)
    plt.plot(df_battles.index, df_battles['cumulative'], color = 'black', label = 'Cumulative WARG', zorder = 1)
#     plt.scatter(df_battles.index, df_battles['cumulative'], s = 100, zorder = 2)
    plt.xticks(df_battles.index, df_battles['Battle'], rotation = 'vertical')

    plt.title(commander + ' ({} - {} - {})'.format(wins, draws, losses))
    plt.tight_layout()
    
    plt.savefig('{}_warg.png'.format(commander))

In [ ]:
df_run = df_run.dropna(subset = ['belligerent'])

In [ ]:
battle_list = []
graph('Moshe Dayan')

In [ ]:
df_battles

In [ ]:
df_model[df_model.Battle == 'Battle_of_Borodino'].transpose()

In [ ]:
df_model[df_model.Battle == 'Six Day War']

In [ ]:
df_battles

In [ ]:
df_war = pd.DataFrame(war_list).sort_values('WAR', ascending = False).reset_index(drop = True)

In [ ]:
df_war.head()

In [ ]:
df_totalwar = df_totalwar[df_totalwar.General != 'Capital punishment']
df_totalwar = df_totalwar[df_totalwar.General != 'Lieutenant general']

In [ ]:
df_totalwar[df_totalwar.General == 'Erwin Rommel']

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(df_totalwar['WAR'], kde = False, bins = 30)
plt.title("Distribution of Generals' War")
plt.ylabel('Number of Generals')
plt.annotate('Napoleon', xy = (16.713, 1), xytext = (17.713, 400), arrowprops = dict(facecolor = 'black'))
plt.annotate('Julius Caesar', xy = (7.364545, 1), xytext = (8.364545, 400), arrowprops = dict(facecolor = 'black'))
plt.tight_layout()
plt.savefig('/home/ethanarsht/Downloads/distplot.png')

In [129]:
def index():
    output_file('docs/index.html')
    source = ColumnDataSource(data = dict(battles = df_totalwar.Battles,
                                          war = df_totalwar.WAR,
                                          general = df_totalwar.General))

    hover = HoverTool(tooltips = [('General', '@general'),
                                  ('Battles', '@battles'),
                                  ('WAR', '@war'),
                                      ])
    p = figure(title = 'WAR vs Number of Battles',
               plot_height = 600, 
               plot_width = 800,

               tools = [hover, "tap", 'box_zoom', ResetTool()])
    p.circle(x = 'war', y = 'battles', size = 10, source = source)
    p.xaxis.axis_label = 'WAR'
    p.yaxis.axis_label = 'Battles'

    link = "{}.html".format('@general')

    if link == 'https://ethanarsht.github.io/military_rankings/Thomas de Courtenay, 5th/13th Earl of Devon':
            link = 'https://ethanarsht.github.io/military_rankings/Thomas de Courtenay'
    url = link
    taptool = p.select(type=TapTool)
    taptool.callback = OpenURL(url=url)
    save(p)

In [130]:
index()

In [153]:
df_battles

,Battle,General,Outcome,Value,Year,cumulative
0,Battle of Arras,Harold Franklyn,D,-0.430169,1940.0,-0.430169


In [112]:
def bokeh(commander):
    global df_battles
    
    df_battles = pd.DataFrame()
    commander_war(commander)
    
#     df_battles = df_battles[df_battles.General == commander]
    df_battles.Year = df_battles.Year.fillna(-5000)
    df_battles.Year = df_battles.Year.astype(float, errors = 'ignore')
    df_battles = df_battles.sort_values('Year')
    wl = pd.DataFrame(df_battles.Outcome.value_counts())
    
    if 'V' in wl.index:
        wins = wl.loc['V', 'Outcome']
    else:
        wins = 0
    if 'D' in wl.index:
        losses = wl.loc['D', 'Outcome']
    else:
        losses = 0
    if 'I' in wl.index:
        draws = wl.loc['I', 'Outcome']
    else:
        draws = 0
    df_battles = df_battles.reset_index(drop = True)
    for i, row in df_battles.iterrows():
        df_battles.ix[i, 'cumulative'] = sum(df_battles.Value[0:i+1])
    
    
    source = ColumnDataSource(data = dict(battle = df_battles.Battle,
                              general = df_battles.General,
                              outcome = df_battles.Outcome,
                              value = df_battles.Value,
                              year = df_battles.Year,
                              cumulative = df_battles.cumulative))
    if all(x != -5000 for x in df_battles.Year):
        hover = HoverTool(tooltips = [('Battle', '@battle'),
                                  ('Outcome', '@outcome'),
                                  ('WAR', '@value'),
                                  ('Cumulative WAR', '@cumulative'),
                                  ('Year', '@year')])
    else:
        hover = HoverTool(tooltips = [('Battle', '@battle'),
                                  ('Outcome', '@outcome'),
                                  ('WAR', '@value'),
                                  ('Cumulative WAR', '@cumulative')
                                  ])
    
    x = list(df_battles.Battle)
    y = list(df_battles.Value)
    y_cum = list(df_battles.cumulative)
    
    if commander == 'Thomas de Courtenay, 5th/13th Earl of Devon':
        commander = 'Thomas de Courtenay'
    
    output_file('docs/{}.html'.format(commander))
#     output_file('{}.html'.format(commander))
    
    p = figure(title = commander + ' ({} - {} - {})'.format(wins, draws, losses), 
               plot_width=800, 
               plot_height=600, 
               x_range = x, 
               tools = [hover, "tap"])
#     p.xaxis.axis_label = "General's Biography: en.wikipedia.org/wiki/{}".format(commander)
    
    p.line(x = 'battle', y = 'cumulative', source = source, line_width = 5)
#     url = 'en.wikipedia.org/wiki/{}'.format(commander)
#     taptool = p.select(type=TapTool)
#     taptool.callback = OpenURL(url=url)
    
    color_mapper = CategoricalColorMapper(factors=['V', 'D', 'I'], palette=['Blue', 'Red', 'Yellow'])
    p.vbar(x = 'battle', 
           top = 'value', 
           source = source, 
           width = 0.9, 
           color={'field': 'outcome', 'transform': color_mapper},
           alpha = 0.5)
    url = 'https://en.wikipedia.org/wiki/@battle'
    taptool = p.select(type=TapTool)
    taptool.callback = OpenURL(url=url)
    
    p.xaxis.major_label_orientation = math.pi/2
    

#     show(p)
    save(p)

In [194]:
battle_list = []
bokeh('Napoleon')

16.678282131


/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [160]:
df_battles

,Battle,General,Outcome,Value,Year,cumulative
0,Siege of Toulon,Napoleon,V,0.479531,1793.0,0.479531
1,Battle of Montenotte,Napoleon,V,0.478016,1796.0,0.957547
2,Battle of Castiglione,Napoleon,V,0.489183,1796.0,1.446729
3,Battle of Lodi,Napoleon,V,0.473926,1796.0,1.920655
4,Battle of Bassano,Napoleon,V,0.468786,1796.0,2.389441
5,Battle of the Bridge of Arcole,Napoleon,V,0.507813,1796.0,2.897254
6,Battle of Rivoli,Napoleon,V,0.518113,1797.0,3.415367
7,Battle of the Pyramids,Napoleon,V,0.518009,1798.0,3.933376
8,Siege of El Arish,Napoleon,V,0.572390,1799.0,4.505766
9,Siege of Acre (1799),Napoleon,V,0.578863,1799.0,5.084629


In [113]:

count = 0
for general in df_run.belligerent.unique()[1:]:
    battle_list = []
    count += 1
    print(str(count) + '/' + str(len(df_run.belligerent.unique())))
    print(general)
    bokeh(general)

1/6620
David Ben-Gurion
0.986016936844


/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


2/6620
Yisrael Galili
0.471721466161
3/6620
Yaakov Dori
0.471721466161
4/6620
Yigael Yadin
0.471721466161
5/6620
Mickey Marcus
0.471721466161
6/6620
Yigal Allon
0.471721466161
7/6620
Yitzhak Rabin
1.03976631482
8/6620
David Shaltiel
0.471721466161
9/6620
Moshe Dayan
2.11324527998
10/6620
Shimon Avidan
0.471721466161
11/6620
Moshe Carmel
0.471721466161
12/6620
Yitzhak Sadeh
0.471721466161
13/6620
Abdul Rahman Hassan Azzam
-0.460186422825
14/6620
Farouk of Egypt
-0.460186422825
15/6620
Abdullah I of Jordan
-0.460186422825
16/6620
Muzahim al-Pachachi
-0.460186422825
17/6620
Husni al-Za'im
-0.460186422825
18/6620
Amin al-Husseini
-0.460186422825
19/6620
Ahmed Ali al-Mwawi
-0.460186422825
20/6620
Mohammed Naguib
-0.460186422825
21/6620
John Bagot Glubb
-0.460186422825
22/6620
Habis al-Majali
-0.460186422825
23/6620
Hasan Salama
-0.460186422825
24/6620
Arab Liberation Army
-0.460186422825
25/6620
Fawzi al-Qawuqji
-0.460186422825
26/6620
Menachem Begin
-0.0610185789382
27/6620
Ariel Sharon
2.

-0.832035073868
213/6620
Asakura Yoshikage
-1.35682614218
214/6620
Timur
0.480766586629
215/6620
Shahrukh Mirza
0.480766586629
216/6620
Khalil Sultan
0.480766586629
217/6620
Miran Shah
0.480766586629
218/6620
Abu Bakr (Timurid)
0.480766586629
219/6620
Sultan Huseyn
0.480766586629
220/6620
Pir Muhammad ibn Jahangir
0.480766586629
221/6620
Taj al-Din Shah-i Shahan Abu'l Fath
0.480766586629
222/6620
Bayezid I
0.000824814520108
223/6620
Süleyman Çelebi
-0.469206810986
224/6620
Mehmed I
-0.469206810986
225/6620
Stefan Lazarević
0.0185150653547
226/6620
Gannys
0.5
227/6620
Heraclius
-0.655036801018
228/6620
Shahrbaraz
0.430540269565
229/6620
Shahin Vahmanzadegan
0.5
230/6620
Eslamu
-0.576241023749
231/6620
Fatagar
-0.576241023749
232/6620
Theophilos (emperor)
-0.507564734422
233/6620
Theophobos
-0.507564734422
234/6620
Manuel the Armenian
-0.507564734422
235/6620
Khaydhar ibn Kawus al-Afshin
0.519145247928
236/6620
Umar al-Aqta
0.519145247928
237/6620
Jaysh ibn Samsama
0.5
238/6620
Damian Da

419/6620
Gilbert Motier de La Fayette
0.450801692212
420/6620
List of Marshals of France
0.450801692212
421/6620
Thomas of Lancaster, 1st Duke of Clarence
-0.43935275921
422/6620
Thomas Beaufort, Duke of Exeter
-0.43935275921
423/6620
John Beaufort, 1st Duke of Somerset
-0.43935275921
424/6620
Zhang Fei
1.05676209115
425/6620
Zhang He
-0.0132155967224
426/6620
Baldwin IV of Jerusalem
0.0533893179319
427/6620
Baldwin of Ibelin
0
428/6620
Balian of Ibelin
-0.965534912969
429/6620
Hugh II of Saint Omer
0
430/6620
Gökböri
0.900345300962
431/6620
Charles I of Naples
0.5
432/6620
Manfred, King of Sicily
-0.5
433/6620
Quintus Fulvius Flaccus (consul 237 BC)
-0.036510668752
434/6620
Hanno the Elder
-0.970611389035
435/6620
Tiberius Sempronius Gracchus (consul 215 BC)
0.506360199087
436/6620
Manius Curius Dentatus
0.5
437/6620
John II Komnenos
0.5
438/6620
Telerig of Bulgaria
-0.419213127244
439/6620
Seron (Syrian governor)
-0.555432379831
440/6620
Judas Maccabeus
2.34634642976
441/6620
Cestius

...

5704/6620
Central Command (India)
0.475283872408
5705/6620
Sagat Singh
0.475283872408
5706/6620
IV Corps (India)
0.475283872408
5707/6620
Tapishwar Narain Raina
0.475283872408
5708/6620
II Corps (India)
0.475283872408
5709/6620
Sartaj Singh (general)
0.475283872408
5710/6620
XV Corps (India)
0.434800640995
5711/6620
I Corps (India)
0.434800640995
5712/6620
Depinder Singh
0.475283872408
5713/6620
XII Corps (India)
0.475283872408
5714/6620
J. F. R. Jacob
0.475283872408
5715/6620
Chief of staff
0.475283872408
5716/6620
Om Prakash Malhotra
0.475283872408
5717/6620
Sardarilal Mathradas Nanda
0.975283872408
5718/6620
Chief of the Naval Staff (India)
0.934800640995
5719/6620
Air chief marshal
0.854624796766
5720/6620
Pratap Chandra Lal
0.475283872408
5721/6620
Chief of the Air Staff (India)
0.434800640995
5722/6620
R. N. Kao
0.475283872408
5723/6620
Research and Analysis Wing
0.475283872408
5724/6620
Tajuddin Ahmad
0.475283872408
5725/6620
Prime Minister of Bangladesh
0.475283872408
5726/6620

5890/6620
Ahmed Haji Hassan
-0.496344868893
5891/6620
Abdullahi Ahmed Irro
-0.496344868893
5892/6620
Ali Hussein
-0.496344868893
5893/6620
Farah Handulle
-0.496344868893
5894/6620
Yussuf Salhan
-0.496344868893
5895/6620
Mohamed Nur Galaal
-0.496344868893
5896/6620
Ali Ismail
-0.496344868893
5897/6620
Abdulrahman Aare
-0.496344868893
5898/6620
Mengistu Haile Mariam
0.507933439066
5899/6620
Demissie Bulto
0.507933439066
5900/6620
Vasiliy Petrov (military)
0.507933439066
5901/6620
Arnaldo Ochoa
0.507933439066
5902/6620
Salim Rubai Ali
0.507933439066
5903/6620
Simon Spoor
1.03136002619
5904/6620
Sukarno
-0.5
5905/6620
Mohammad Hatta
-0.5
5906/6620
Abdul Haris Nasution
-0.5
5907/6620
Sudirman
-0.519801581683
5908/6620
Alvin Spoor
0.531360026185
5909/6620
Hubertus van Mook
0.531360026185
5910/6620
Dan K. McNeill
0.477204898395
5911/6620
Jacko Page
0.477204898395
5912/6620
Dadullah
-0.46565395979
5913/6620
Abdul Qassim
-0.46565395979
5914/6620
Franklin L. Hagenbeck
0.409690856686
5915/6620
Ro

6090/6620
Nisar Ahmed
-0.408576664448
6091/6620
Ismail al-Sallabi
0.5
6092/6620
Osama Arusi
0.5
6093/6620
Isatai'i
-0.5
6094/6620
Quanah Parker
-0.5
6095/6620
Big Red Meat
-0.5
6096/6620
White Horse (chief)
-0.5
6097/6620
Big Bow (chief)
-0.5
6098/6620
Ayad Futayyih Al-Rawi
0.42506530044
6099/6620
Hussein Rashid al-Tikriti
0.42506530044
6100/6620
Esmael Shorabi
-0.413777537074
6101/6620
Mariano Osorio
0.522849786628
6102/6620
José Ordóñez
0.522849786628
6103/6620
James B. Steedman
0.603478464355
6104/6620
Georg Stumme
-0.462331528158
6105/6620
Wilhelm Ritter von Thoma
-0.462331528158
6106/6620
Robert Hoke
-0.486600264382
6107/6620
William Lamb (Confederate States Army officer)
-0.486600264382
6108/6620
Colonel Van Den Brinck
-0.509316359521
6109/6620
Henrique Dias
0.520894563343
6110/6620
António Filipe Camarão
0.520894563343
6111/6620
André Vidal de Negreiros
0.520894563343
6112/6620
Trevylyan Napier
0.0263183587297
6113/6620
William Pakenham (Royal Navy officer)
0.0263183587297
6114/

-0.423061579285
6295/6620
Oh Eung-jeong
-0.423061579285
6296/6620
Im Hyeon
-0.423061579285
6297/6620
Yi Deok-hoe
-0.423061579285
6298/6620
Jeong Gi-won
-0.423061579285
6299/6620
Hwang Dae-jung
-0.423061579285
6300/6620
Shin Ho
-0.423061579285
6301/6620
Jo Gyeong-nam
-0.423061579285
6302/6620
Yang Hao (Ming dynasty)
-0.942444025283
6303/6620
Jiang Biao
-0.423061579285
6304/6620
Mao Cheng Xian
-0.423061579285
6305/6620
Aristagoras
-0.5
6306/6620
Megabates
-0.5
6307/6620
Manuel Boutoumites
0.454705972728
6308/6620
Charles V, Holy Roman Emperor
-0.5
6309/6620
Charles III, Duke of Savoy
-0.5
6310/6620
Suganuma Sadamichi
-0.393763261162
6311/6620
Satake Yoshiaki
-0.56979709213
6312/6620
Ōta Sukemasa
-0.56979709213
6313/6620
Nagano Narimasa
-0.56979709213
6314/6620
Oda Ujiharu
-0.56979709213
6315/6620
Utsunomiya Hirotsuna
-0.56979709213
6316/6620
Satomi Yoshihiro
-0.56979709213
6317/6620
Nasu Suketane
-0.56979709213
6318/6620
Yura Narishige
-0.56979709213
6319/6620
Mita Tsunahide
-0.569797092

6482/6620
Pavel Nakhimov
-0.458843396349
6483/6620
Anthonie II Schetz
-0.438351325737
6484/6620
Sigismund III Vasa
0.441572118448
6485/6620
Mikhail Shein
-0.430173985905
6486/6620
Jean-Jacques Uhrich — Wikipédia
-0.466616226028
6487/6620
Jean François Leval
-0.550712928396
6488/6620
Francisco Copons
0.562154062134
6489/6620
John Byrne Skerrett
0.562154062134
6490/6620
Gastone Gambara
-0.507421111176
6491/6620
Leslie Morshead
0.519001801467
6492/6620
Ronald Scobie
0.519001801467
6493/6620
Jean François Carteaux
0.486829850046
6494/6620
Jean François Cornu de La Poype
0.486829850046
6495/6620
Marquess of Normanby
-0.475257727116
6496/6620
Jan Piotr Sapieha
-0.567953100004
6497/6620
Aleksander Józef Lisowski
-0.567953100004
6498/6620
Grigory Dolgorukov
0.579290596326
6499/6620
Aleksey Golokhvastov
0.579290596326
6500/6620
Tang Ching-sung
-0.585563210798
6501/6620
Marc-Edmond Dominé
0.596766683407
6502/6620
William Oliphant (died after 1313)
-0.5
6503/6620
Sanada Yukitaka
0.418396670321
65

In [ ]:
mka = df_run.loc[8965, 'belligerent']

In [ ]:
mka

In [ ]:
df_battles

In [ ]:
battle_list = []
bokeh(u'Ariel Sharon')

In [ ]:
df_warchart = pd.DataFrame({'Labels': ['Win = 1', 'Replacement-Level Performance = .51', 'WAR From Battle = .49'],
              'Values': [1, .51, .49]})

In [ ]:
df_warchart = df_warchart.sort_values('Values', ascending = True)

In [ ]:
df_warchart

In [ ]:
df_warchart = df_warchart.iloc[::-1]

In [ ]:
fig = plt.figure(figsize = (12,9))
plt.style.use('fivethirtyeight')
plt.bar(df_warchart.index, df_warchart.Values, width = 1, color = ['red', 'blue', 'yellow'])
plt.xticks(df_warchart.index + .5, df_warchart.Labels, rotation = 45)
plt.xticks()
plt.title("Bonaparte's WAR in Battle of Borodino")
plt.tight_layout()
plt.savefig('napoleon_war_sample')

# ax.set_axis_bgcolor('gray')

In [ ]:
df_warchart_two = pd.DataFrame({'Labels': ['Win = -1', 'Replacement-Level Performance = .49', 'WAR From Battle = -.51'],
              'Values': [-1, -.49, -.51]})

In [ ]:
fig = plt.figure(figsize = (12,9))
plt.style.use('fivethirtyeight')
plt.bar(df_warchart_two.index, df_warchart_two.Values, width = 1, color = ['red', 'blue', 'white'])
plt.xticks(df_warchart_two.index, df_warchart_two.Labels, rotation = 45)
plt.title("Kutuzov's WAR in Battle of Borodino")
plt.tight_layout()
plt.savefig('/home/ethanarsht/Downloads/kutuzov_war_sample')

# ax.set_axis_bgcolor('gray')